In [2]:
import pandas as pd
import json

with open('data_full.json') as json_file: 
    data_dict = json.load(json_file) 

train_data = data_dict['train']
val_data = data_dict['val']
test_data = data_dict['test']

oos_train = data_dict['oos_train']
oos_val = data_dict['oos_val']
oos_test = data_dict['oos_test']


train_df = pd.DataFrame(train_data, columns =['query', 'intent'])
val_df = pd.DataFrame(val_data, columns =['query', 'intent'])
test_df = pd.DataFrame(test_data, columns =['query', 'intent'])

#print(train_df.head())
# grouping
#train_df.to_excel('ds_grouped.xlsx',sheet_name="Sheet1")
#train_df.groupby("intent").count().to_excel('ds_grouped.xlsx',sheet_name="Sheet1")

In [3]:
print(oos_train[:2])

[['how much is an overdraft fee for bank', 'oos'], ['why are exponents preformed before multiplication in the order of operations', 'oos']]
